In [1]:
print("Jai Shree Ram!!")

Jai Shree Ram!!


In [11]:
from key import google_api_key

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("new project.pdf")
data = loader.load()


In [6]:
len(data)

31

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter(chunk_size = 1000)
docs = text_spliter.split_documents(data)

print("Total number of documents :: ",len(docs))

Total number of documents ::  48


In [12]:
from dotenv import load_dotenv
load_dotenv()

False

In [15]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=google_api_key)
vector = embedding.embed_query("Har Har Mahadev!")
vector[:5]


[0.04371386393904686,
 -0.02195916138589382,
 -0.014615615829825401,
 0.0026698028668761253,
 0.030525753274559975]

In [16]:
vectorstore = Chroma.from_documents(documents=docs,embedding=embedding)

In [18]:
retriver = vectorstore.as_retriever(search_type = "similarity",search_kwags = {"k":10})

k= retriver.invoke("what is the title of document")

In [19]:
print(k[1].page_content)

ii 
 
TABLE OF CONTENTS 
PRELIMINARY PAGES 
List of Tables ....................................................... i 
Declaration ............................................................. vi 
 Certificate ............................................................ vii  
Abstract .............................................................. viii  
List of Figures ...................................................... ix  
CHAPTER 1: INTRODUCTION ....................................................... 1-3 
1.1 Background  
 1.1.1 Digital Marketplace Evolution  
1.1.2 Current State of Code Commerce  
1.2 Problem Statement 
1.2.1 Developer Monetization Challenges 
 1.2.2 Code Quality Assurance Issues  
1.2.3 Market Integration Gaps 
 1.3 Project Objectives  
1.3.1 Platform Development Goals  
1.3.2 User Experience Targets  
1.3.3 Security and Quality Standards  
1.4 Project Scope 
CHAPTER 2: LITERATURE REVIEW  ................................... 4-5


In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temprature = 0.4,max_tokens=500,google_api_key = google_api_key)

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = ("You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}")

prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
])

In [22]:
q_a_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriver,q_a_chain)

In [23]:
response = rag_chain.invoke({"input":"what is the name of application?"})
print(response['answer'])

The application's name is CodeMart.
